# DynamoDB Client Examples

A low-level client representing Amazon DynamoDB  https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/dynamodb.html#client

In [1]:
import boto3
client=boto3.client('dynamodb') # create the DDB client

## Create Table

In [2]:
response = client.create_table(
    TableName='ratings',
    KeySchema=[
        {
            'AttributeName': 'userId',
            'KeyType': 'HASH'  #Partition key
        },
        {
            'AttributeName': 'movieId',
            'KeyType': 'RANGE'  #Sort key
        }
        
    ],
    AttributeDefinitions=[
        {
            'AttributeName': 'userId',
            'AttributeType': 'N'
        },
        {
            'AttributeName': 'movieId',
            'AttributeType': 'N'
        }
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

In [3]:
response

{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'movieId',
    'AttributeType': 'N'},
   {'AttributeName': 'userId', 'AttributeType': 'N'}],
  'TableName': 'ratings',
  'KeySchema': [{'AttributeName': 'userId', 'KeyType': 'HASH'},
   {'AttributeName': 'movieId', 'KeyType': 'RANGE'}],
  'TableStatus': 'CREATING',
  'CreationDateTime': datetime.datetime(2020, 8, 23, 20, 16, 16, 669000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 5,
   'WriteCapacityUnits': 5},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:226595923650:table/ratings',
  'TableId': 'e54240e4-5073-4c4e-a627-08e7f14e6e2f'},
 'ResponseMetadata': {'RequestId': 'VIVOM5OJGCETK7VDCM47KMGKMVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Aug 2020 20:16:16 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '576',
   'connection': 'keep-

## Insert Item

In [14]:
import pandas as pd
from decimal import Decimal
df=pd.read_csv('./ratings_1000.csv')
for i in df.index:
    userid,movieid,rating,time=df.iloc[i]
    try:
       # print('userid is {}, and movieId is {}'.format(userid,movieid))
        #print(userid)
        
        client.put_item(
            TableName='ratings',
            Item={
                'userId': {'N': str(userid)},
                'movieId':{'N': str(movieid)},
                
                'rating': {'N':str(rating)},
                'timestamp':{'S':str(time)}
        })
        #print('write one record {}'.format(i))
    except Exception as e:
        print(e)

## Get Item

In [17]:
response=client.get_item(
    TableName='ratings',
    Key={
        'userId':{'N':'7'},
        'movieId':{'N':'7'}
    }
    )
response['Item']

{'movieId': {'N': '7'},
 'rating': {'N': '3'},
 'userId': {'N': '7'},
 'timestamp': {'S': '1011208220.0'}}

## Update Item

In [29]:
response=client.update_item(
    TableName='ratings',
    Key={
        'userId':{'N':'7'},
        'movieId':{'N':'7'}
    },
    UpdateExpression='SET #att=:val',
    ConditionExpression='#att < :val2',
    ExpressionAttributeNames={'#att':'rating'},
    ExpressionAttributeValues={':val':{'N':'3'}, ':val2':{'N':'5'}},

    ReturnValues='ALL_NEW'
)
response

{'Attributes': {'movieId': {'N': '7'},
  'rating': {'N': '3'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011208220.0'}},
 'ResponseMetadata': {'RequestId': '5HR582U8IHHEVBN1O8AQECGSSNVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Aug 2020 22:21:06 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '107',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5HR582U8IHHEVBN1O8AQECGSSNVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '177573034'},
  'RetryAttempts': 0}}

## Delete Item

In [30]:
response=client.delete_item(
    TableName='ratings',
    Key={
        'userId':{'N':'7'},
        'movieId':{'N':'7'}
    }
)

In [31]:
response

{'ResponseMetadata': {'RequestId': 'NG9C2JK8T5DQ2VBEQ93QMLTPAFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 23 Aug 2020 22:23:06 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'NG9C2JK8T5DQ2VBEQ93QMLTPAFVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

## Query

In [33]:
response=client.query(
    TableName='ratings',
    KeyConditionExpression='userId=:uid AND movieId BETWEEN :mid1 AND :mid2',
    ExpressionAttributeValues={
        ':uid':{'N':'7'},
        ':mid1':{'N':'1'},
        ':mid2':{'N':'100'}
    }
)
response['Items']

[{'movieId': {'N': '3'},
  'rating': {'N': '3'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011208463.0'}},
 {'movieId': {'N': '11'},
  'rating': {'N': '4'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011207889.0'}},
 {'movieId': {'N': '15'},
  'rating': {'N': '2'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011208732.0'}},
 {'movieId': {'N': '16'},
  'rating': {'N': '3'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011205378.0'}},
 {'movieId': {'N': '17'},
  'rating': {'N': '2'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011207676.0'}},
 {'movieId': {'N': '24'},
  'rating': {'N': '3'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011207238.0'}}]

## Scan

In [35]:
response=client.scan(
    TableName='ratings',
    # IndexName=,
    # ExclusiveStartKey= ,The primary key of the first item that this operation will evaluate
)
response['Items'][:5] # show beginning 5

[{'movieId': {'N': '3'},
  'rating': {'N': '3'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011208463.0'}},
 {'movieId': {'N': '11'},
  'rating': {'N': '4'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011207889.0'}},
 {'movieId': {'N': '15'},
  'rating': {'N': '2'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011208732.0'}},
 {'movieId': {'N': '16'},
  'rating': {'N': '3'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011205378.0'}},
 {'movieId': {'N': '17'},
  'rating': {'N': '2'},
  'userId': {'N': '7'},
  'timestamp': {'S': '1011207676.0'}}]

## Delete Table

In [ ]:
response = client.delete_table(
    TableName='ratings'
)